In [9]:
import ipynb_importer
import tensorflow_5_5_mnist_inference as mnist_inference
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

BATCH_SIZE=100
LEARNING_RATE_BASE=0.8
LEARNING_RATE_DECAY=0.99
REGULARAZTION_RATE=0.0001
TRAINING_STEPS=30000
MOVING_AVERAGE_DECAY=0.99

MODEL_SAVE_PATH='./tmp/best_example/'
MODEL_NAME='model.ckpt'

def train(mnist):
    x=tf.placeholder(tf.float32,[None,mnist_inference.INPUT_NODE],name='x-input')
    y_=tf.placeholder(tf.float32,[None,mnist_inference.OUTPUT_NODE],name='y-input')
    
    regularizer=tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    
    #向前传播
    y=mnist_inference.inference(x,regularizer)
    global_step=tf.Variable(0,trainable=False)
    
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    loss=cross_entropy_mean+tf.add_n(tf.get_collection('losses'))
    learning_rate=tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples/BATCH_SIZE,
        LEARNING_RATE_DECAY
    )
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
        
    saver=tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(TRAINING_STEPS):
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            _,loss_value,step=sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})
            
            #每1000轮保存一次模型
            if i % 1000 ==0:
                print 'After %d training steps,loss=%f' % (step,loss_value)
                
                #保存模型
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=step)
                
                
def main(argv=None):
    mnist=input_data.read_data_sets('./MNIST_data',one_hot=True)
    train(mnist)
    
if __name__=='__main__':
    """
    ValueError: Variable layer1/weights already exists, disallowed. 
    Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:
    """
    tf.get_variable_scope().reuse_variables()
    
    tf.app.run()
            
         

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training steps,loss=3.760038
After 1001 training steps,loss=0.468802
After 2001 training steps,loss=0.426766
After 3001 training steps,loss=0.207890
After 4001 training steps,loss=0.140158
After 5001 training steps,loss=0.123904
After 6001 training steps,loss=0.156228
After 7001 training steps,loss=0.122877
After 8001 training steps,loss=0.102882
After 9001 training steps,loss=0.118275
After 10001 training steps,loss=0.111574
After 11001 training steps,loss=0.106098
After 12001 training steps,loss=0.114823
After 13001 training steps,loss=0.091506
After 14001 training steps,loss=0.128570
After 15001 training steps,loss=0.090939
After 16001 training steps,loss=0.140453
After 17001 training steps,loss=0.109333
After 18001 training steps,loss=0.095992
After 19001 training steps,loss=

SystemExit: 